# Data Preprocessing of Ground Truth

Import statments. Ensure that the package 'endolas' that is shipped with this notebook can be imported. 

In [ ]:
import imageio
import numpy as np
import json
import os
import random

from matplotlib import pyplot as plt
from skimage.transform import warp
from skimage.morphology import flood
from skimage.filters import gaussian
from glob import glob
from shutil import copyfile

from endolas import utils

# Functionality
This functionality block contains functionality needed to create the data sets. 4 Different files are created per sample point. Those are:
- Endoscopic image of the current frame
- Segmentation mask
- File containing the x- and y-coordinates of each keypoint
- Moving image. That is an imaginary camera projection.

In [ ]:
def create_and_store(base_dir, save_dir, sources, image_index, height, width, sigma, sigma_back):
    """ Extract all frames from .avi and keypoints from .rois.
        Store single images, masks, moving images and keypoints.
    """
    
    image_index_2_roi_2_xy = dict()
    for video in sources:
        video_path = os.path.join(base_dir, "{}.avi".format(video))
        rois_path = os.path.join(base_dir, "{}.rois".format(video))

        reader = imageio.get_reader(video_path)

        file = open(rois_path)
        roi_data = json.load(file)

        for frame, image in enumerate(reader):  
            # Extract image
            save_path = os.path.join(save_dir, "{}.png".format(image_index))
            plt.imsave(save_path, image, cmap="gray")

            rois = roi_data["frames"][frame]["roi_positions"]
            roi_2_xy = dict()

            # Create mask
            mask = np.zeros(image.shape)
            for roi in rois:
                if not roi['placed']:
                    continue

                id = roi['id']
                x = roi['pos']['x']
                y = roi['pos']['y']

                x = int(round(roi['pos']['x'], 0))
                y = int(round(roi['pos']['y'], 0))

                roi_2_xy[id] = (x, y)
                mask[y][x] =  255

            mask = gaussian(mask,sigma=0.9)
            mask = (mask / mask.max()) * 255
            mask = np.where(mask>0.1, 1.0, 0.0)

            plt.imshow(mask, cmap="gray")
            mask_path = os.path.join(save_dir, "{}_m.png".format(image_index))
            plt.imsave(mask_path, mask, cmap="gray")
            
            # Create keypoints .json
            json_path = os.path.join(save_dir, "{}.json".format(image_index))
            json_file = json.dumps(roi_2_xy)

            f = open(json_path, "w")
            f.write(json_file)
            f.close() 
            file.close()
            
            # Create moving image
            moving = np.zeros((height, width))
            for key, value in roi_2_xy.items():
                moving[value[1]][value[0]] = 1
            moving = utils.apply_smoothing(moving, sigma=2, sigma_back=15)
            save_path = save_dir + os.sep + "{}_mov.png".format(image_index)
            plt.imsave(save_path, moving, cmap='gray')

            image_index += 1

# Parameters
Parameters used for the specific data set.

In [ ]:
# Parameters for the image space
height = 768
width = 768

# Amount of smoothing
sigma = 2

# Amount of smoothing for the background
sigma_back= 15

# Training Set
Here the training set is create. Ensure the reuired directories are created.

In [ ]:
image_index = 0
base_dir = "Data/LASTEN/raw"
save_dir = "Data/LASTEN/train"
sources = ["KK2_gap0_Cam_16904_Cine2_0_19",
           "KK4_gap0_Cam_16904_Cine2_0_19",
           "KK11_gap0_Cam_16904_Cine3_0_19",
           "KK16_gap0_Cam_16904_Cine2_0_19",
           "KK17_gap0_Cam_16904_Cine2_0_19",
           "KK18_gap0_Cam_16904_Cine2_0_19",
           "KK19_gap0_Cam_16904_Cine2_0_19",
           "KK24_gap0_Cam_16904_Cine2_0_19"]

create_and_store(base_dir, save_dir, sources, image_index, height, width, sigma, sigma_back)

# Validation Set
Here the validation set is create. Ensure the reuired directories are created.

In [ ]:
image_index = 160
base_dir = "Data/LASTEN/raw"
save_dir = "Data/LASTEN/validation"
sources = ["KK3_gap0_Cam_16904_Cine2_0_19",
           "KK9_gap0_Cam_16904_Cine2_0_19"]

create_and_store(base_dir, save_dir, sources, image_index, height, width, sigma, sigma_back)

# Test Set
Here the test set is create. Ensure the reuired directories are created.

In [ ]:
image_index = 200
base_dir = "Data/LASTEN/raw"
save_dir = "Data/LASTEN/test"
sources = ["KK5_gap0_Cam_16904_Cine2_0_19",
           "KK20_gap0_Cam_16904_Cine2_0_19"]

create_and_store(base_dir, save_dir, sources, image_index, height, width, sigma, sigma_back)

# Fixed Image

Create a fixed image with specified shape. That is the imaginary laser projection.

In [ ]:
width = 768
height = 768

n_x = 18
n_y = 18

spacing_x = 20
spacing_y = 20

offset_x = (width - ((n_x - 1) * spacing_x)) / 2.0
offset_y = (height - ((n_y - 1) * spacing_y)) / 2.0

sigma = (2 / 32) * spacing_x
sigma_back = (15 / 32) * spacing_y

In [ ]:
def get_fixed_data(sample_index):  
    fixed = np.zeros((height, width))
    index_2_xy = dict()
    index = 0
    for index_y in reversed(range(n_y)):
        y = int(round(offset_y + (spacing_y * index_y)))

        for index_x in range(n_x):
            x = int(round(offset_x + (spacing_x * index_x)))
            
            fixed[y][x] = 1
            index_2_xy[index] = (x, y)
            
            index += 1
            
    return fixed, index_2_xy

In [ ]:
fixed, index_2_xy = get_fixed_data(0)

padding = 30
fixed = np.pad(fixed, padding, mode='constant')
fixed = utils.apply_smoothing(fixed, sigma=sigma, sigma_back=sigma_back)
fixed = fixed[padding:height+padding, padding:width+padding]

save_des = "Data/LASTEN/fix"
file = "_{}_{}".format(width, spacing_x)
plt.imsave(save_des + file + ".png", fixed, cmap='gray', vmin=0)

json_file = json.dumps(index_2_xy)
f = open(save_des + file + ".json", "w")
f.write(json_file)
f.close()